# 🎯 Phase-D External data downloads 

This notebook prepares the external datasets needed for Phase D validation.

D**esign goals**
- Provide short manual instructions where licence acceptance is required.
- Automate downloads where a stable direct link exists.
- Standardise folder names and file paths so later notebooks can run without guesswork.
- Cache first behaviour, reruns should not re download if outputs exist.

**Datasets covered**
1. OS Open Greenspace polygons (manual download, then code handles extraction and normalisation)
2. LAEI 2019 Focus Areas (automated download and extraction)
3. PTAL Grid Values (automated download and extraction)
4. OSM major roads cache for Camden and Islington (generated from your existing OSM PBF)

# ❗Note: D1 is manual download step❗

### D0 Setup

In [2]:

# ============================================================
# D0 Setup and paths
# ============================================================

import os
import re
import json
import time
import shutil
import zipfile
import hashlib
from pathlib import Path
from datetime import datetime, timezone

import requests

def now_utc_iso():
    return datetime.now(timezone.utc).isoformat()

def sha256_file(path: Path) -> str:
    h = hashlib.sha256()
    with Path(path).open("rb") as f:
        for chunk in iter(lambda: f.read(1024 * 1024), b""):
            h.update(chunk)
    return h.hexdigest()

NOTEBOOK_DIR = Path(".").resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent

DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
RESULTS_DIR = DATA_DIR / "results"

RAW_DIR.mkdir(parents=True, exist_ok=True)

# Core pipeline inputs (these should already exist from Phase A to C)
VENUES_CLEAN = PROCESSED_DIR / "venues_clean.geojson"
OSM_PBF = RAW_DIR / "osm" / "greater-london-260110.osm.pbf"

# External datasets for Phase D
GREENS_DIR = RAW_DIR / "os_open_greenspace"
LAEI_DIR = RAW_DIR / "laei_focus_areas"
PTAL_DIR = RAW_DIR / "ptal"
MAJOR_ROADS_DIR = RAW_DIR / "osm_major_roads"

GREENS_DIR.mkdir(parents=True, exist_ok=True)
LAEI_DIR.mkdir(parents=True, exist_ok=True)
PTAL_DIR.mkdir(parents=True, exist_ok=True)
MAJOR_ROADS_DIR.mkdir(parents=True, exist_ok=True)

# Outputs written by this notebook
GREENS_EXTRACT_DIR = GREENS_DIR / "opgrsp_essh_tq_extracted"
LAEI_EXTRACT_DIR = LAEI_DIR / "laei_gis"
PTAL_EXTRACT_DIR = PTAL_DIR / "ptal_grid_values"
MAJOR_ROADS_GPKG = MAJOR_ROADS_DIR / "major_roads_camden_islington.gpkg"

# Provenance logs
PROV_DIR = RAW_DIR / "provenance_phaseD_downloads"
PROV_DIR.mkdir(parents=True, exist_ok=True)

PROV_GREENS = PROV_DIR / "provenance_os_open_greenspace.json"
PROV_LAEI = PROV_DIR / "provenance_laei_focus_areas.json"
PROV_PTAL = PROV_DIR / "provenance_ptal.json"
PROV_ROADS = PROV_DIR / "provenance_osm_major_roads.json"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("RAW_DIR:", RAW_DIR)
print("VENUES_CLEAN:", VENUES_CLEAN)
print("OSM_PBF:", OSM_PBF)
print("GREENS_DIR:", GREENS_DIR)
print("LAEI_DIR:", LAEI_DIR)
print("PTAL_DIR:", PTAL_DIR)
print("MAJOR_ROADS_DIR:", MAJOR_ROADS_DIR)


PROJECT_ROOT: /home/sai/test/notebook_test
RAW_DIR: /home/sai/test/notebook_test/data/raw
VENUES_CLEAN: /home/sai/test/notebook_test/data/processed/venues_clean.geojson
OSM_PBF: /home/sai/test/notebook_test/data/raw/osm/greater-london-260110.osm.pbf
GREENS_DIR: /home/sai/test/notebook_test/data/raw/os_open_greenspace
LAEI_DIR: /home/sai/test/notebook_test/data/raw/laei_focus_areas
PTAL_DIR: /home/sai/test/notebook_test/data/raw/ptal
MAJOR_ROADS_DIR: /home/sai/test/notebook_test/data/raw/osm_major_roads


### D1 OS Open Greenspace polygons (Manual download)
Used as an independent, authoritative greenspace layer for external validation of GreenScore at matched spatial support. Manual download is required due to licence acceptance and the absence of a stable public API.

**Steps to download:**
- Open the official OS Data Hub download page: https://osdatahub.os.uk/data/downloads/open/OpenGreenspace
- In Specify an area: Select TQ (100 km tile covering Greater London)
- In Data format : Choose ESRI Shapefile
- Click Download and move the zip file at: `./data/raw/os_open_greenspace`
- Note: do not unzip, just move the zip file to `./data/raw/os_open_greenspace`


Expected result after running the next cell
- A clean extracted folder: `data/raw/os_open_greenspace/opgrsp_essh_tq_extracted/`
- A detected GreenspaceSite shapefile path printed in the output


In [4]:
# ============================================================
# D1.1 Data preparation: OS Open Greenspace (manual download)
# Normalise to a stable folder name and stable shapefile paths
# ============================================================

GREENS_DIR.mkdir(parents=True, exist_ok=True)

# The downloaded zip name can vary. Prefer a stable name if you use one.
PREFERRED_ZIP = GREENS_DIR / "opgrsp_essh_tq.zip"

def pick_greens_zip() -> Path:
    if PREFERRED_ZIP.exists() and PREFERRED_ZIP.stat().st_size > 0:
        return PREFERRED_ZIP
    zips = sorted([p for p in GREENS_DIR.glob("*.zip") if p.stat().st_size > 0])
    if not zips:
        raise FileNotFoundError(
            "Missing OS Open Greenspace zip.\n"
            "Download from OS Data Hub (tile TQ, ESRI Shapefile) and place the zip into:\n"
            f"  {GREENS_DIR}\n"
            "Optional: rename it to opgrsp_essh_tq.zip for clarity."
        )
    return zips[0]

ZIP_PATH = pick_greens_zip()

# If final folder missing, create it from extraction
if not GREENS_EXTRACT_DIR.exists():
    tmp = GREENS_DIR / "_tmp_extract"
    if tmp.exists():
        shutil.rmtree(tmp)
    tmp.mkdir(parents=True, exist_ok=True)

    print("Extracting:", ZIP_PATH.name)
    with zipfile.ZipFile(ZIP_PATH, "r") as z:
        z.extractall(tmp)

    # If there is a single top folder, move it. Otherwise keep the tmp content.
    top_dirs = [p for p in tmp.iterdir() if p.is_dir()]
    if len(top_dirs) == 1:
        shutil.move(str(top_dirs[0]), str(GREENS_EXTRACT_DIR))
        shutil.rmtree(tmp)
    else:
        shutil.move(str(tmp), str(GREENS_EXTRACT_DIR))

else:
    print("Greenspace extract folder already exists:", GREENS_EXTRACT_DIR)

# Validate we can locate the GreenspaceSite shapefile
expected = GREENS_EXTRACT_DIR / "data" / "TQ_GreenspaceSite.shp"
if expected.exists():
    SITE_SHP = expected
else:
    candidates = list(GREENS_EXTRACT_DIR.rglob("*GreenspaceSite*.shp"))
    if not candidates:
        raise FileNotFoundError(
            "Could not find GreenspaceSite shapefile after extraction.\n"
            f"Searched under: {GREENS_EXTRACT_DIR}"
        )
    SITE_SHP = candidates[0]

prov = {
    "created_at_utc": now_utc_iso(),
    "dataset": "OS Open Greenspace (tile TQ, ESRI Shapefile)",
    "method": "manual_download_then_extract",
    "zip_used": str(ZIP_PATH),
    "zip_sha256": sha256_file(ZIP_PATH),
    "extract_dir": str(GREENS_EXTRACT_DIR),
    "greenspace_site_shp": str(SITE_SHP)
}
PROV_GREENS.write_text(json.dumps(prov, indent=2), encoding="utf-8")

print("OS Open Greenspace ready")
print("Extracted folder:", GREENS_EXTRACT_DIR)
print("GreenspaceSite shapefile:", SITE_SHP)
print("Provenance:", PROV_GREENS)


Greenspace extract folder already exists: /home/sai/test/notebook_test/data/raw/os_open_greenspace/opgrsp_essh_tq_extracted
OS Open Greenspace ready
Extracted folder: /home/sai/test/notebook_test/data/raw/os_open_greenspace/opgrsp_essh_tq_extracted
GreenspaceSite shapefile: /home/sai/test/notebook_test/data/raw/os_open_greenspace/opgrsp_essh_tq_extracted/data/TQ_GreenspaceSite.shp
Provenance: /home/sai/test/notebook_test/data/raw/provenance_phaseD_downloads/provenance_os_open_greenspace.json


### D2 LAEI 2019 Focus Areas (Automation download)
Independent policy layer used for convergent validation. ComfortScore is expected to be lower inside LAEI focus areas.

**Steps to download:**
- Open the official website: https://data.london.gov.uk/dataset/london-atmospheric-emissions-inventory-laei-2019-air-quality-foc-2zj76#
- Look for GIS file (2. GIS files  (40.61 kB))
- Direct download link : https://data.london.gov.uk/download/2zj76/36d88eac-849b-400a-ae44-2dc93bdb966d/2.%20GIS%20files.zip


In [5]:
# ============================================================
# D2.1 Download and extract: LAEI 2019 Focus Areas
# Robust to zip structure changes, extracts into a stable folder
# ============================================================

LAEI_ZIP = LAEI_DIR / "laei_2019_focus_areas.zip"

LAEI_URL = (
    "https://data.london.gov.uk/download/2zj76/"
    "36d88eac-849b-400a-ae44-2dc93bdb966d/"
    "2.%20GIS%20files.zip"
)

def download_file(url: str, out_path: Path, timeout_s: int = 90):
    out_path = Path(out_path)
    if out_path.exists() and out_path.stat().st_size > 0:
        return "already_present"
    print("Downloading:", url)
    with requests.get(url, stream=True, timeout=timeout_s) as r:
        r.raise_for_status()
        with out_path.open("wb") as f:
            for chunk in r.iter_content(chunk_size=1024 * 256):
                if chunk:
                    f.write(chunk)
    if out_path.stat().st_size == 0:
        raise RuntimeError("Downloaded file is empty")
    return "downloaded"

status = download_file(LAEI_URL, LAEI_ZIP)
print("LAEI zip status:", status, "|", LAEI_ZIP)

if not LAEI_EXTRACT_DIR.exists():
    LAEI_EXTRACT_DIR.mkdir(parents=True, exist_ok=True)
    print("Extracting into:", LAEI_EXTRACT_DIR)
    with zipfile.ZipFile(LAEI_ZIP, "r") as z:
        z.extractall(LAEI_EXTRACT_DIR)
else:
    print("LAEI extract folder already exists:", LAEI_EXTRACT_DIR)

# Find a shapefile to confirm content
shps = sorted(LAEI_EXTRACT_DIR.rglob("*.shp"))
if not shps:
    raise FileNotFoundError("LAEI shapefile not found after extraction")

# Prefer a file that looks like Focus Areas if available
preferred = None
for p in shps:
    if "focus" in p.name.lower() and "area" in p.name.lower():
        preferred = p
        break
LAEI_SHP = preferred if preferred else shps[0]

prov = {
    "created_at_utc": now_utc_iso(),
    "dataset": "LAEI 2019 Air Quality Focus Areas",
    "method": "direct_download_then_extract",
    "dataset_page": "https://data.london.gov.uk/dataset/london-atmospheric-emissions-inventory-laei-2019-air-quality-foc-2zj76#",
    "direct_url": LAEI_URL,
    "zip_path": str(LAEI_ZIP),
    "zip_sha256": sha256_file(LAEI_ZIP),
    "extract_dir": str(LAEI_EXTRACT_DIR),
    "shapefile_selected": str(LAEI_SHP)
}
PROV_LAEI.write_text(json.dumps(prov, indent=2), encoding="utf-8")

print("LAEI Focus Areas ready")
print("Extracted folder:", LAEI_EXTRACT_DIR)
print("Selected shapefile:", LAEI_SHP)
print("Provenance:", PROV_LAEI)


Downloading: https://data.london.gov.uk/download/2zj76/36d88eac-849b-400a-ae44-2dc93bdb966d/2.%20GIS%20files.zip
LAEI zip status: downloaded | /home/sai/test/notebook_test/data/raw/laei_focus_areas/laei_2019_focus_areas.zip
Extracting into: /home/sai/test/notebook_test/data/raw/laei_focus_areas/laei_gis
LAEI Focus Areas ready
Extracted folder: /home/sai/test/notebook_test/data/raw/laei_focus_areas/laei_gis
Selected shapefile: /home/sai/test/notebook_test/data/raw/laei_focus_areas/laei_gis/2. GIS files/LAEI2019_FocusAreas.shp
Provenance: /home/sai/test/notebook_test/data/raw/provenance_phaseD_downloads/provenance_laei_focus_areas.json


### D3 PTAL Public Transport Accessibility Levels (Automation download)

A diagnostic accessibility proxy.
Used cautiously to illustrate construct mismatch with the wheelchair tagging.

 - Dataset page : https://data.london.gov.uk/dataset/public-transport-accessibility-levels-24rz6/
 - Download : PTAL Grid Values.zip  (8.59 MB)
 - Extract at : data/raw/ptal/
 - Direct download link : https://data.london.gov.uk/download/24rz6/514d2847-94a8-4b9d-8a70-fdded01719a0/2015%20%20PTALs%20Grid%20Values.zip



In [6]:
# ============================================================
# D3.1 Download and extract: PTAL Grid Values
# ============================================================

PTAL_ZIP = PTAL_DIR / "ptal_grid_values.zip"

PTAL_URL = (
    "https://data.london.gov.uk/download/24rz6/"
    "514d2847-94a8-4b9d-8a70-fdded01719a0/"
    "2015%20%20PTALs%20Grid%20Values.zip"
)

status = download_file(PTAL_URL, PTAL_ZIP)
print("PTAL zip status:", status, "|", PTAL_ZIP)

if not PTAL_EXTRACT_DIR.exists():
    PTAL_EXTRACT_DIR.mkdir(parents=True, exist_ok=True)
    print("Extracting into:", PTAL_EXTRACT_DIR)
    with zipfile.ZipFile(PTAL_ZIP, "r") as z:
        z.extractall(PTAL_EXTRACT_DIR)
else:
    print("PTAL extract folder already exists:", PTAL_EXTRACT_DIR)

# Confirm expected MapInfo TAB exists somewhere
tabs = sorted(PTAL_EXTRACT_DIR.rglob("*.TAB")) + sorted(PTAL_EXTRACT_DIR.rglob("*.tab"))
PTAL_TAB = tabs[0] if tabs else None

prov = {
    "created_at_utc": now_utc_iso(),
    "dataset": "PTAL Grid Values",
    "method": "direct_download_then_extract",
    "dataset_page": "https://data.london.gov.uk/dataset/public-transport-accessibility-levels-24rz6/",
    "direct_url": PTAL_URL,
    "zip_path": str(PTAL_ZIP),
    "zip_sha256": sha256_file(PTAL_ZIP),
    "extract_dir": str(PTAL_EXTRACT_DIR),
    "tab_file_detected": str(PTAL_TAB) if PTAL_TAB else None
}
PROV_PTAL.write_text(json.dumps(prov, indent=2), encoding="utf-8")

print("PTAL ready")
print("Extracted folder:", PTAL_EXTRACT_DIR)
print("Detected TAB:", PTAL_TAB)
print("Provenance:", PROV_PTAL)


Downloading: https://data.london.gov.uk/download/24rz6/514d2847-94a8-4b9d-8a70-fdded01719a0/2015%20%20PTALs%20Grid%20Values.zip
PTAL zip status: downloaded | /home/sai/test/notebook_test/data/raw/ptal/ptal_grid_values.zip
Extracting into: /home/sai/test/notebook_test/data/raw/ptal/ptal_grid_values
PTAL ready
Extracted folder: /home/sai/test/notebook_test/data/raw/ptal/ptal_grid_values
Detected TAB: /home/sai/test/notebook_test/data/raw/ptal/ptal_grid_values/2015  PTALs Contours 280515.TAB
Provenance: /home/sai/test/notebook_test/data/raw/provenance_phaseD_downloads/provenance_ptal.json


### D4 OSM major roads cache for Camden and Islington

This is generated from your existing Geofabrik Greater London PBF.
Uses the venue footprint to define a buffered AOI, then extracts only major highway classes.

Output: `data/raw/osm_major_roads/major_roads_camden_islington.gpkg`


In [7]:
# ============================================================
# D4.1 Build OSM major roads cache as a GeoPackage (UPDATED)
# Fix: pyrosm expects bounding_box as a list or Shapely Polygon, not a tuple
# This cell does not download anything
# ============================================================

import json
from datetime import datetime, timezone
from pathlib import Path

import geopandas as gpd
from shapely.geometry import box

try:
    import pyrosm
    PYROSM_OK = True
except Exception as e:
    PYROSM_OK = False
    PYROSM_ERR = str(e)

OUT_LAYER = "major_roads"
AOI_BUFFER_M = 2500
REFRESH_MAJOR_ROADS = False

major_highways = [
    "motorway", "motorway_link",
    "trunk", "trunk_link",
    "primary", "primary_link",
    "secondary", "secondary_link",
    "tertiary", "tertiary_link",
]

def now_utc_iso():
    return datetime.now(timezone.utc).isoformat()

# Checks
if not OSM_PBF.exists():
    raise FileNotFoundError(
        "Missing OSM PBF. Expected:\n"
        f"  {OSM_PBF}\n"
        "Place the Greater London .osm.pbf into data/raw/osm/ then rerun."
    )

if not VENUES_CLEAN.exists():
    raise FileNotFoundError(
        "Missing venues_clean.geojson. Expected:\n"
        f"  {VENUES_CLEAN}\n"
        "Run Phase A first, then rerun."
    )

if not PYROSM_OK:
    raise RuntimeError(
        "pyrosm is not installed in this environment.\n"
        "Install it, then rerun:\n"
        "  pip install pyrosm\n"
        f"Import error: {PYROSM_ERR}"
    )

if MAJOR_ROADS_GPKG.exists() and MAJOR_ROADS_GPKG.stat().st_size > 0 and (not REFRESH_MAJOR_ROADS):
    print("Major roads cache already present:", MAJOR_ROADS_GPKG)
else:
    print("Reading venues and building AOI")

    venues = gpd.read_file(VENUES_CLEAN)
    if venues.crs is None:
        raise ValueError("venues_clean.geojson has no CRS")

    venues_27700 = venues.to_crs("EPSG:27700")
    try:
        geom = venues_27700.geometry.union_all()
    except Exception:
        geom = venues_27700.unary_union

    aoi_27700 = geom.convex_hull.buffer(float(AOI_BUFFER_M))

    # Convert AOI to WGS84
    aoi_wgs = gpd.GeoSeries([aoi_27700], crs="EPSG:27700").to_crs("EPSG:4326").iloc[0]
    minx, miny, maxx, maxy = aoi_wgs.bounds

    # pyrosm accepts a list [minx, miny, maxx, maxy] OR a shapely polygon
    bbox_list = [float(minx), float(miny), float(maxx), float(maxy)]
    bbox_poly = box(*bbox_list)

    print("AOI bbox WGS84:", tuple(bbox_list))
    print("Extracting roads using pyrosm")

    # Use polygon form for maximum compatibility
    osm = pyrosm.OSM(str(OSM_PBF), bounding_box=bbox_poly)

    roads = osm.get_network(
        network_type="driving",
        extra_attributes=["highway", "name", "ref"],
    )

    if roads is None or len(roads) == 0:
        raise RuntimeError("pyrosm returned 0 road edges inside the AOI")

    if roads.crs is None:
        roads = roads.set_crs("EPSG:4326", allow_override=True)

    roads = roads[roads["highway"].astype(str).isin(major_highways)].copy()
    if len(roads) == 0:
        raise RuntimeError("0 major road features after highway class filter")

    # Clip in EPSG:27700 for stable metric geometry
    roads_27700 = roads.to_crs("EPSG:27700")
    roads_27700 = gpd.clip(roads_27700, aoi_27700)
    roads_27700 = roads_27700[roads_27700.geometry.notna() & (~roads_27700.geometry.is_empty)].copy()

    keep_cols = [c for c in ["highway", "name", "ref", "geometry"] if c in roads_27700.columns]
    roads_27700 = roads_27700[keep_cols].copy()

    if len(roads_27700) == 0:
        raise RuntimeError("0 road features remain after AOI clip")

    if MAJOR_ROADS_GPKG.exists():
        try:
            MAJOR_ROADS_GPKG.unlink()
        except Exception:
            pass

    roads_27700.to_file(MAJOR_ROADS_GPKG, layer=OUT_LAYER, driver="GPKG")

    prov = {
        "created_at_utc": now_utc_iso(),
        "dataset": "OSM major roads (derived)",
        "method": "pyrosm_from_existing_pbf",
        "input_osm_pbf": str(OSM_PBF),
        "input_venues": str(VENUES_CLEAN),
        "aoi_buffer_m": float(AOI_BUFFER_M),
        "aoi_bbox_wgs84": {"minx": float(minx), "miny": float(miny), "maxx": float(maxx), "maxy": float(maxy)},
        "major_highway_classes": major_highways,
        "output_gpkg": str(MAJOR_ROADS_GPKG),
        "output_layer": OUT_LAYER,
        "output_crs": "EPSG:27700",
        "n_features": int(len(roads_27700)),
    }
    PROV_ROADS.write_text(json.dumps(prov, indent=2), encoding="utf-8")

    print("Saved:", MAJOR_ROADS_GPKG, "| rows:", len(roads_27700))
    print("Provenance:", PROV_ROADS)


Reading venues and building AOI
AOI bbox WGS84: (-0.24825063505740577, 51.490368436869396, -0.04099452782589048, 51.59920692189553)
Extracting roads using pyrosm


/home/sai/miniconda3/envs/dft/lib/python3.11/site-packages/pyrosm/networks.py:37: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  edges, nodes = prepare_geodataframe(


Saved: /home/sai/test/notebook_test/data/raw/osm_major_roads/major_roads_camden_islington.gpkg | rows: 12321
Provenance: /home/sai/test/notebook_test/data/raw/provenance_phaseD_downloads/provenance_osm_major_roads.json


### D5 Checklist and input audit

- It reports what exists and what is missing.
- It only hard fails if the core pipeline inputs are missing.
- Missing external datasets are reported as missing, not treated as a crash.


In [8]:
# ============================================================
# D5.1 Checklist and input audit
# ============================================================

import pandas as pd

def file_status(p: Path) -> dict:
    p = Path(p)
    exists = p.exists()
    is_dir = exists and p.is_dir()
    is_file = exists and p.is_file()
    size_mb = None
    if is_file:
        size_mb = round(p.stat().st_size / (1024 * 1024), 3)
    return {
        "path": str(p),
        "exists": bool(exists),
        "type": "dir" if is_dir else "file",
        "size_mb": size_mb,
    }

items = [
    ("required", "venues_clean.geojson", VENUES_CLEAN),
    ("required", "greater-london osm.pbf", OSM_PBF),

    ("external", "OS Open Greenspace extract folder", GREENS_EXTRACT_DIR),
    ("external", "LAEI focus areas extract folder", LAEI_EXTRACT_DIR),
    ("external", "PTAL extract folder", PTAL_EXTRACT_DIR),
    ("external", "OSM major roads gpkg", MAJOR_ROADS_GPKG),

    ("provenance", "OS Greenspace provenance", PROV_GREENS),
    ("provenance", "LAEI provenance", PROV_LAEI),
    ("provenance", "PTAL provenance", PROV_PTAL),
    ("provenance", "Major roads provenance", PROV_ROADS),
]

rows = []
for group, name, path in items:
    st = file_status(path)
    rows.append({"group": group, "name": name, **st})

status_df = pd.DataFrame(rows).sort_values(["group", "name"]).reset_index(drop=True)
display(status_df)

missing_required = status_df[(status_df["group"] == "required") & (status_df["exists"] == False)]
if len(missing_required) > 0:
    raise FileNotFoundError(
        "Missing required core inputs:\n" + "\n".join(missing_required["path"].tolist())
    )

print("Project root:", PROJECT_ROOT)


,group,name,path,exists,type,size_mb
0,external,LAEI focus areas extract folder,/home/sai/test/notebook_test/data/raw/laei_foc...,True,dir,NaN
1,external,OS Open Greenspace extract folder,/home/sai/test/notebook_test/data/raw/os_open_...,True,dir,NaN
2,external,OSM major roads gpkg,/home/sai/test/notebook_test/data/raw/osm_majo...,True,file,3.551
3,external,PTAL extract folder,/home/sai/test/notebook_test/data/raw/ptal/pta...,True,dir,NaN
4,provenance,LAEI provenance,/home/sai/test/notebook_test/data/raw/provenan...,True,file,0.001
5,provenance,Major roads provenance,/home/sai/test/notebook_test/data/raw/provenan...,True,file,0.001
6,provenance,OS Greenspace provenance,/home/sai/test/notebook_test/data/raw/provenan...,True,file,0.001
7,provenance,PTAL provenance,/home/sai/test/notebook_test/data/raw/provenan...,True,file,0.001
8,required,greater-london osm.pbf,/home/sai/test/notebook_test/data/raw/osm/grea...,True,file,115.500
9,required,venues_clean.geojson,/home/sai/test/notebook_test/data/processed/ve...,True,file,0.875


Project root: /home/sai/test/notebook_test
